In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from Classes.CartTreeClassifier import CartTreeClassifier
from Classes.CartTreeTeacher import CartTreeTeacher

from Functions.metrics import *
from Functions.split_functions import *
from Functions.spreading_functions import *
from Functions.search_split_functions import *

from Functions.metrics import accuracy
from sys import setrecursionlimit
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler

from Classes.LayerGen import LayerGenerator
from Classes.Layer import Layer
from Classes.Perceptron import Perceptron
from Classes.PerceptronBWSearcher import PerceptronBWSearcher

from Functions.function_of_activation import sigmoid, dsigmoid
from Functions.activation_functions import activation_with_cach, standart_activation, dactivation_by_ddiases, dactivation_by_dweights
from Functions.lossFunctions import euclideError
from Functions.errorFunctions import MSE


In [2]:
data = pd.read_csv('../Data/DataLaba2/neo_task_smoted.csv')
mm_scaler = MinMaxScaler()
data_scaled = mm_scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)
data_scaled_x, hazardous = SMOTE().fit_resample(data_scaled.drop(['hazardous'], axis=1), data['hazardous'])
data_scaled_x = pd.DataFrame(data_scaled_x, columns=data.columns[data.columns != 'hazardous'])
X_train, X_test, y_train, y_test = train_test_split(data_scaled, hazardous, test_size=0.5, random_state=42)


In [3]:
layer_mass = LayerGenerator(sizes_of_input_layer=X_train.shape[1],
                            sizes_of_hidden_layers=[X_train.shape[1] // 2, X_train.shape[1] // 3],
                            sizes_of_output_layer=1,
                            activation_functions=[sigmoid,
                                                  sigmoid,
                                                  sigmoid,
                                                  sigmoid],
                            dactivation_functions=[dsigmoid,
                                                   dsigmoid,
                                                   dsigmoid,
                                                   dsigmoid],
                            function_of_generation=lambda a, b: np.random.rand(a, b)).generate()
layer_mass


In [4]:
def batch_spliter(x, y, size, num):
    batches_x = list()
    batches_y = list()
    for _ in range(num):
        args = np.arange(x.shape[0])
        np.random.shuffle(args)
        batches_x.append(x[args][:size])
        batches_y.append(y[args][:size])
    return zip(batches_x, batches_y)


In [5]:
def MSE(predicted, real):
    return np.sum((real-predicted)**2, axis=1)/real.shape[0]


In [6]:
def dMSE(predicted, real):
    return 2 * (real-predicted)/real.shape[0]


In [7]:
batch_split = list(batch_spliter(X_train[:50].to_numpy(), y_train[:50].to_numpy(), 10, 10))
batch_split


[(array([[2.81682125e-02, 1.31853220e-01, 1.29346701e-01, 3.23320818e-01,
          1.00000000e+00],
         [4.15094849e-03, 1.75600018e-01, 9.96763689e-01, 4.96497077e-01,
          1.00000000e+00],
         [3.64125835e-04, 2.59016470e-01, 5.49847114e-01, 7.13391740e-01,
          0.00000000e+00],
         [1.60130765e-04, 2.97577045e-01, 4.67405724e-03, 7.83062161e-01,
          0.00000000e+00],
         [5.64704079e-04, 1.53028201e-01, 7.39863349e-02, 6.75010430e-01,
          0.00000000e+00],
         [5.55587736e-03, 2.08811630e-01, 8.42641715e-01, 4.70171047e-01,
          0.00000000e+00],
         [3.11830608e-02, 2.81861689e-01, 6.54633082e-01, 3.14763960e-01,
          1.00000000e+00],
         [3.34653745e-03, 2.32008584e-01, 1.57611532e-01, 5.06883605e-01,
          1.00000000e+00],
         [1.20293952e-03, 1.49744938e-01, 2.72760081e-01, 6.07843137e-01,
          0.00000000e+00],
         [9.70155104e-03, 1.35212048e-01, 3.95044003e-01, 4.19818091e-01,
          1.00000

In [8]:
perceptron = PerceptronBWSearcher(layers=layer_mass, 
                                  activation_function=activation_with_cach, 
                                  dactivation_function_by_dweights=dactivation_by_dweights,
                                  dactivation_function_by_ddisases=dactivation_by_ddiases,
                                  derror_function=dMSE, 
                        batch_spliter_function=lambda x, y: batch_spliter(x, y, 10, 10), 
                                  error_function=MSE,
                        n_steps=10)


In [9]:
perceptron.search(X_train[:50].to_numpy(), y_train[:50].to_numpy())


ValueError: shapes (10,1) and (10,1) not aligned: 1 (dim 1) != 10 (dim 0)